`BcForms` is a toolkit for unambiguously describing biocomplexes. `BcForms` represents biocomplexes as a set of the subunits and their stoichiometries and a set of crosslinks linking the subunits.
The subunits can be further described by `BpForms` or `openbabel`.
This jupyter notebook illustrates how to use the `BcForms` Python API with some simple and complex examples. Please see the [documentation](https://docs.karrlab.org/bcforms/) for more information.

### Import `BcForms` and dependencies

In [1]:
import bcforms
import bpforms
import openbabel
from wc_utils.util.chem import OpenBabelUtils

## Simple `BcForms` examples
This section illustrates how to use the BcForm natation with simple examples.

### Create the `BcForms` object with subunits but no crosslinks
In this example, we will describe a homodimer of subunit unit_1, where unit_1 is a peptide AAA.

In [2]:
bc_form_1 = bcforms.BcForm().from_str('2 * unit_1')
bc_form_1.set_subunit_attribute('unit_1', 'structure', bpforms.ProteinForm().from_str('AAA'))

#### Validate the `BcForms` object
We can validate the `BcForms` object. The function returns a list of errors.

In [3]:
bc_form_1.validate()

[]

#### Calculate physical properties and structure of the `Bcforms` object
Having defined the biocomplex as a `BcForms` object, we can calculate properties and structure of the complex.

In [4]:
str(bc_form_1.get_formula())

'C18H36N6O8'

In [5]:
bc_form_1.get_mol_wt()

464.52000000000004

In [6]:
bc_form_1.get_charge()

2

In [7]:
OpenBabelUtils.export(bc_form_1.get_structure(), format='smiles', options=[])

'C[C@H]([NH3+])C(=O)N[C@@H](C)C(=O)N[C@@H](C)C(=O)O.C[C@H]([NH3+])C(=O)N[C@@H](C)C(=O)N[C@@H](C)C(=O)O'

#### Check if two biocomplexes are equal
We can also check whether two `BcForms` objects are semantically equal.

In [8]:
bc_form_1_t = bcforms.BcForm().from_str('unit_1 + unit_1')
bc_form_1_t.set_subunit_attribute('unit_1', 'structure', bpforms.ProteinForm().from_str('AAA'))
bc_form_1.is_equal(bc_form_1_t)

True

In [9]:
bc_form_1_f = bcforms.BcForm().from_str('3 * unit_1')
bc_form_1_f.set_subunit_attribute('unit_1', 'structure', bpforms.ProteinForm().from_str('AAA'))
bc_form_1.is_equal(bc_form_1_f)

False

### Create the `BcForms` object with subunits and crosslinks
In this example, we will describe a homodimer of subunit unit_2, where unit_2 is a single amino acid Cysteine, and that the two subunits are bonded by a disulfide bond between the Cysteines.

In [10]:
bc_form_2 = bcforms.BcForm().from_str('2 * unit_2'
                                      '| crosslink: [ left-bond-atom: unit_2(1)-1S11 |'
                                                    ' left-displaced-atom: unit_2(1)-1H11-1 |'
                                                    ' right-bond-atom: unit_2(2)-1S11 |'
                                                    ' right-displaced-atom: unit_2(2)-1H11-1 ]')
bc_form_2.set_subunit_attribute('unit_2', 'structure', bpforms.ProteinForm().from_str('C'))

#### Validate the `BcForms` object

In [11]:
bc_form_2.validate()

[]

#### Calculate physical properties and structure of the `Bcforms` object

In [12]:
str(bc_form_2.get_formula())

'C6H14N2O4S2'

In [13]:
bc_form_2.get_mol_wt()

242.308

In [14]:
bc_form_2.get_charge()

4

In [15]:
OpenBabelUtils.export(bc_form_2.get_structure(), format='smiles', options=[])

'OC(=O)[C@@H]([NH3+])CSSC[C@@H](C(=O)O)[NH3+]'

## Real `BcForms` examples
This section uses a few real protein complex to demonstrate BcForm

### Disulfide bonds
One common type of intermolecular crosslink is disulfide bond. In proteins, disulfide bonds can form between Cysteine residues within a chain or between chains. [This](https://www.uniprot.org/help/disulfid) is a link to the UniProt disulfide bond page for more information. `BcForms` is capable of capturing the interchain disulfide bonds. Here, we  demonstrate the `BcForms` representation of disulfide bonds with three examples: a parallel homodimer, an anti-parallel homodimer, and a heterodimer.

#### Parallel homodimer
This example is [Bone morphogenetic protein 2-A](https://www.uniprot.org/uniprot/P25703) in *Xenopus laevis*. Two bmp-2a units form a disulfide-linked homodimer at C-362. Because many proteins such as bmp2-a go through post-translational processing, we first use a helper function to get the `BpForms` representation of the chain in the mature protein.

In [16]:
def get_chain(fasta, chain_idx):
    chain_fasta = fasta[chain_idx[0]-1:chain_idx[1]]
    chain = bpforms.ProteinForm().from_str(fasta)
    return chain

In [17]:
p25703_fasta = ('MVAGIHSLLLLLFYQVLLSGCTGLIPEEGKRKYTESGRSSPQQSQRVLNQFELRLLSMFG'
                'LKRRPTPGKNVVIPPYMLDLYHLHLAQLAADEGTSAMDFQMERAASRANTVRSFHHEESM'
                'EEIPESREKTIQRFFFNLSSIPNEELVTSAELRIFREQVQEPFESDSSKLHRINIYDIVK'
                'PAAAASRGPVVRLLDTRLVHHNESKWESFDVTPAIARWIAHKQPNHGFVVEVNHLDNDKN'
                'VPKKHVRISRSLTPDKDNWPQIRPLLVTFSHDGKGHALHKRQKRQARHKQRKRLKSSCRR'
                'HPLYVDFSDVGWNDWIVAPPGYHAFYCHGECPFPLADHLNSTNHAIVQTLVNSVNTNIPK'
                'ACCVPTELSAISMLYLDENEKVVLKNYQDMVVEGCGCR')
p25703_chain_idx = (285,398)
p25703_chain = get_chain(p25703_fasta, p25703_chain_idx)

In [18]:
str(p25703_chain.get_formula())

'C2023H3233N591O536S16'

In [19]:
p25703_chain.get_mol_wt()

44923.678

Then, we can represent the complex in `BcForm`

In [20]:
str_bmp2a = ('2 * p25703'
             ' | crosslink: [ left-bond-atom: p25703(1)-{}S11 |'
                            ' right-bond-atom: p25703(2)-{}S11 |' 
                            ' left-displaced-atom: p25703(1)-{}H11-1 |'
                            'right-displaced-atom: p25703(2)-{}H11-1 ]'.format( \
                            362-p25703_chain_idx[0], 362-p25703_chain_idx[0], \
                            362-p25703_chain_idx[0], 362-p25703_chain_idx[0]))
bc_form_bmp2a = bcforms.BcForm().from_str(str_bmp2a)
bc_form_bmp2a.set_subunit_attribute('p25703', 'structure', p25703_chain)

With this representation, we can calculate the properties of the complex

In [21]:
str(bc_form_bmp2a.get_formula())

'C4046H6464N1182O1072S32'

In [22]:
bc_form_bmp2a.get_mol_wt()

89845.34

As shown, the formula of the complex is 2 hydrogen atoms less than 2 times the formula of the subunit, and the molecular weight is approximately 2 less than 2 times that of the subunit.

#### Anti-parallel homodimer
This example is [Disintegrin schistatin](https://www.uniprot.org/uniprot/P83658) in *Echis carinatus*. The two subunits form a homodimer with two anti-parallel disulfide bonds between C-7 and C-12. Again, we first represent the structure of the subunit using `BpForms`.

In [23]:
p83658_fasta = ('NSVHPCCDPVICEPREGEHCISGPCCENCYFLNSGTICKRARGDGNQDYCTGITPDCPRN'
                'RYNV')
p83658_chain_idx = (1,64)
p83658_chain = get_chain(p83658_fasta, p83658_chain_idx)

In [24]:
str(p83658_chain.get_formula())

'C290H456N91O89S10'

In [25]:
p83658_chain.get_mol_wt()

6961.986

We can then represent the complex using `BcForms`.

In [26]:
str_dids = ('2 * p83658'
            ' | crosslink: [ left-bond-atom: p83658(1)-{}S11 |'
                           ' right-bond-atom: p25703(2)-{}S11 |'
                           ' left-displaced-atom: p25703(1)-{}H11-1 |'
                           ' right-displaced-atom: p25703(2)-{}H11-1 ]'
            ' | crosslink: [ left-bond-atom: p83658(1)-{}S11 |'
                           ' right-bond-atom: p25703(2)-{}S11 |'
                           ' left-displaced-atom: p25703(1)-{}H11-1 |'
                           ' right-displaced-atom: p25703(2)-{}H11-1 ]'.format( \
                            7-p83658_chain_idx[0], 12-p83658_chain_idx[0], \
                            7-p83658_chain_idx[0], 12-p83658_chain_idx[0], \
                            7-p83658_chain_idx[0], 12-p83658_chain_idx[0], \
                            7-p83658_chain_idx[0], 12-p83658_chain_idx[0]))
bc_form_dids = bcforms.BcForm().from_str(str_dids)
bc_form_dids.set_subunit_attribute('p83658', 'structure', p83658_chain)

We can thus compute the physical properties of the complex.

In [27]:
str(bc_form_dids.get_formula())

'C580H908N182O178S20'

In [28]:
bc_form_dids.get_mol_wt()

13919.94

As shown, the formula of the complex is 4 hydrogens less than 2 times the formula of the subunit, and the molecular weight of the complex is approximately 4 less than 2 times that of the subunit.

#### Heterodimer
The third example is a disulfide-linked heterodimer. The protein Snaclec botrocetin in *Bothrops jajaraca* is composed of one [Snaclec botrocetin subunit alpha](https://www.uniprot.org/uniprot/P22029) and one [Snaclec botrocetin subunit beta](https://www.uniprot.org/uniprot/P22030). The two subunits are linked by a disulfide bond between C-80 of P22029 and C-75 OF P22030. First, we get the `BpForms` representation of the two subunits.

In [29]:
p22029_fasta = ('DCPSGWSSYEGNCYKFFQQKMNWADAERFCSEQAKGGHLVSIKIYSKEKDFVGDLVTKNI'
                'QSSDLYAWIGLRVENKEKQCSSEWSDGSSVSYENVVERTVKKCFALEKDLGFVLWINLYC'
                'AQKNPFVCKSPPP')
p22029_chain_idx = (1,133)
p22029_chain = get_chain(p22029_fasta, p22029_chain_idx)

In [30]:
str(p22029_chain.get_formula())

'C682H1048N176O187S8'

In [31]:
p22029_chain.get_mol_wt()

14961.410999999998

In [32]:
p22030_fasta = ('DCPPDWSSYEGHCYRFFKEWMHWDDAEEFCTEQQTGAHLVSFQSKEEADFVRSLTSEMLK'
                'GDVVWIGLSDVWNKCRFEWTDGMEFDYDDYYLIAEYECVASKPTNNKWWIIPCTRFKNFV'
                'CEFQA')
p22030_chain_idx = (1,133)
p22030_chain = get_chain(p22030_fasta, p22030_chain_idx)

In [33]:
str(p22030_chain.get_formula())

'C682H972N166O178S10'

In [34]:
p22030_chain.get_mol_wt()

14664.862000000001

Having defined the two subunits, we can represent the heterodimer as a `BcForms` object.

In [35]:
str_sle = ('p22029 + p22030'
           ' | crosslink: [ left-bond-atom: p22029(1)-{}S11 |'
           ' right-bond-atom: p22030(2)-{}S11 |'
           ' left-displaced-atom: p22029(1)-{}H11-1 |'
           ' right-displaced-atom: p22030(2)-{}H11-1 ]'.format( \
            80-p22029_chain_idx[0], 75-p22030_chain_idx[0], \
            80-p22029_chain_idx[0], 75-p22030_chain_idx[0]))
bc_form_sle = bcforms.BcForm().from_str(str_sle)
bc_form_sle.set_subunit_attribute('p22029', 'structure', p22029_chain)
bc_form_sle.set_subunit_attribute('p22030', 'structure', p22030_chain)

We can compute the formula and molecular weight of the complex.

In [36]:
str(bc_form_sle.get_formula())

'C1364H2018N342O365S18'

In [37]:
bc_form_sle.get_mol_wt()

29624.256999999998

### Crosslinks
There exist many other types of covalent linkages between two proteins that are involved in forming protein complexes. [This](https://www.uniprot.org/help/crosslnk) is a link to the UniProt page on crosslinks for more information. When the linkage information is clearly known from databases, then `BcForms` is capable of concretly desribing the crosslink.

#### Sumoylation
Sumoylation is a common interchain crosslink where the proteins are covalently bound to small SUMO proteins, a process involved in nuclear retention. In this example, we will consider the sumoylation of protein [Chromatin assembly factor 1 subunit A](https://www.uniprot.org/uniprot/Q13111) in human. Sumoylation takes place when the lysine at position 182 in CAF1A_HUMAN forms a [Glycyl lysine isopeptide (Lys-Gly)](https://annotation.dbi.udel.edu/cgi-bin/resid?id=AA0125) bond with the glycine at the C-terminus in SUMO1_HUMAN. To describe the protein complex, we first use `BpForms` to describe the subunits

In [38]:
q13111_fasta = ('MLEELECGAPGARGAATAMDCKDRPAFPVKKLIQARLPFKRLNLVPKGKADDMSDDQGTS'
                'VQSKSPDLEASLDTLENNCHVGSDIDFRPKLVNGKGPLDNFLRNRIETSIGQSTVIIDLT'
                'EDSNEQPDSLVDHNKLNSEASPSREAINGQREDTGDQQGLLKAIQNDKLAFPGETLSDIP'
                'CKTEEEGVGCGGAGRRGDSQECSPRSCPELTSGPRMCPRKEQDSWSEAGGILFKGKVPMV'
                'VLQDILAVRPPQIKSLPATPQGKNMTPESEVLESFPEEDSVLSHSSLSSPSSTSSPEGPP'
                'APPKQHSSTSPFPTSTPLRRITKKFVKGSTEKNKLRLQRDQERLGKQLKLRAEREEKEKL'
                'KEEAKRAKEEAKKKKEEEKELKEKERREKREKDEKEKAEKQRLKEERRKERQEALEAKLE'
                'EKRKKEEEKRLREEEKRIKAEKAEITRFFQKPKTPQAPKTLAGSCGKFAPFEIKEHMVLA'
                'PRRRTAFHPDLCSQLDQLLQQQSGEFSFLKDLKGRQPLRSGPTHVSTRNADIFNSDVVIV'
                'ERGKGDGVPERRKFGRMKLLQFCENHRPAYWGTWNKKTALIRARDPWAQDTKLLDYEVDS'
                'DEEWEEEEPGESLSHSEGDDDDDMGEDEDEDDGFFVPHGYLSEDEGVTEECADPENHKVR'
                'QKLKAKEWDEFLAKGKRFRVLQPVKIGCVWAADRDCAGDDLKVLQQFAACFLETLPAQEE'
                'QTPKASKRERRDEQILAQLLPLLHGNVNGSKVIIREFQEHCRRGLLSNHTGSPRSPSTTY'
                'LHTPTPSEDAAIPSKSRLKRLISENSVYEKRPDFRMCWYVHPQVLQSFQQEHLPVPCQWS'
                'YVTSVPSAPKEDSGSVPSTGPSQGTPISLKRKSAGSMCITQFMKKRRHDGQIGAEDMDGF'
                'QADTEEEEEEEGDCMIVDVPDAAEVQAPCGAASGAGGGVGVDTGKATLTASPLGAS')
q13111_chain_idx = (1,956)
q13111_chain = get_chain(q13111_fasta, q13111_chain_idx)

In [39]:
q13111_chain.get_formula()

AttrDefault(<class 'float'>, False, {'C': 4613.0, 'H': 7563.0, 'N': 1356.0, 'O': 1324.0, 'S': 35.0})

In [40]:
q13111_chain.get_mol_wt()

104328.51500000001

In [41]:
p63165_fasta = ('MSDQEAKPSTEDLGDKKEGEYIKLKVIGQDSSEIHFKVKMTTHLKKLKESYCQRQGVPMN'
                'SLRFLFEGQRIADNHTPKELGMEEEDVIEVYQEQTGGHSTV')
p63165_chain_idx = (2,97)
p63165_chain = get_chain(p63165_fasta, p63165_chain_idx)

In [42]:
p63165_chain.get_formula()

AttrDefault(<class 'float'>, False, {'C': 501.0, 'H': 815.0, 'N': 138.0, 'O': 146.0, 'S': 5.0})

In [43]:
p63165_chain.get_mol_wt()

11268.150999999998

Then, we can define the complex using `BcForms`.

In [44]:
str_sumo = ('q13111 + p63165'
           ' | crosslink: [ left-bond-atom: q13111-{}N1-1 |'
           ' right-bond-atom: p63165-{}C2 |'
           ' left-displaced-atom: q13111-{}H1+1 |'
           ' left-displaced-atom: q13111-{}H1 |' 
           ' right-displaced-atom: p63165-{}O1 |'
           ' right-displaced-atom: p63165-{}H1 ]'.format( \
            182-q13111_chain_idx[0], 97-p63165_chain_idx[0], \
            182-q13111_chain_idx[0], 182-q13111_chain_idx[0], \
            97-p63165_chain_idx[0], 97-p63165_chain_idx[0]))
bc_form_sumo = bcforms.BcForm().from_str(str_sumo)
bc_form_sumo.set_subunit_attribute('q13111', 'structure', q13111_chain)
bc_form_sumo.set_subunit_attribute('p63165', 'structure', p63165_chain)

In [45]:
str(bc_form_sumo.get_formula())

'C5114H8375N1494O1469S40'

In [46]:
bc_form_sumo.get_mol_wt()

115577.64300000001